In [2]:
from Bio import AlignIO, SeqIO, Seq, pairwise2, Align
from Bio.pairwise2 import format_alignment
import pandas as pd
import os
import numpy as np
import time

/home/norbert/miniconda3/lib/python3.11/site-packages/Bio/pairwise2.py:278: BiopythonDeprecationWarning: Bio.pairwise2 has been deprecated, and we intend to remove it in a future release of Biopython. As an alternative, please consider using Bio.Align.PairwiseAligner as a replacement, and contact the Biopython developers if you still need the Bio.pairwise2 module.
  warnings.warn(


In [3]:
'''
uzupelnij do konca tworzenie listy exonow
to samo zrob dla intronow

sklekotaj dwie pozostale tabelki o zadane parametry

dodaj funkcje liczaca wszystkie pliki w folderze oraz dajaca input ile z tych plikow przeszlo pomyslnie procedury liczenia

'''

#dobra
#beda trzy tabelki outputowe, wiec musisz napisac trzy funkcje do robienia tych tabelek. Dlatego pamietaj przy argumentach by dawac odpowiednie zrodla dancyh all fine TS_ TLH_ etc

'\nuzupelnij do konca tworzenie listy exonow\nto samo zrob dla intronow\n\nsklekotaj dwie pozostale tabelki o zadane parametry\n\ndodaj funkcje liczaca wszystkie pliki w folderze oraz dajaca input ile z tych plikow przeszlo pomyslnie procedury liczenia\n\n'

In [24]:

#ASSUMPTIONS
#- both file, path_to_file_before_MAFFT and path_to_file_after_MAFFT have to have the same name
#- they have to be .fasta




min_length_aligned_sequence = 30 #Minimal lenght of sequence which could be an exon
extreme_homology = 0.97 #percentage of homology of sequence, treshold #I assume two faulty aligned nucleotides per 100 (98%) and one more nt because sometimes latest nt can move from end of one sequence to beginning next sequence

#9 plikow
path_to_file_before_MAFFT = "/home/norbert/mrdn/euglena/kod_i_pliki/surowe_pliki_plus_minus_500/raw_reads_9/merging_fastas"
path_to_file_after_MAFFT = "/home/norbert/mrdn/euglena/kod_i_pliki/surowe_pliki_plus_minus_500/raw_reads_9/fastas_after_mafft_na_probe"

#ten niedzialajacy pierwszy LON309
#path_to_file_before_MAFFT = "/home/norbert/mrdn/euglena/kod_i_pliki/surowe_pliki_plus_minus_500/raw_reads_9/roboczy_przed_fastami/"
#path_to_file_after_MAFFT = "/home/norbert/mrdn/euglena/kod_i_pliki/surowe_pliki_plus_minus_500/raw_reads_9/roboczy_po_fastach/"

#pozostale 8
#ath_to_file_before_MAFFT = "/home/norbert/mrdn/euglena/kod_i_pliki/surowe_pliki_plus_minus_500/raw_reads_9/inne_przed_fastami/"
#path_to_file_after_MAFFT = "/home/norbert/mrdn/euglena/kod_i_pliki/surowe_pliki_plus_minus_500/raw_reads_9/inne_po_fastach/"


#DODAJ FRAGMENT MOWIACY ZE JESLI: NA RAZIE SIE WSTRZYMAM, MOZE NIE JEST TO POTRZEBNE
    #LICZBA INTRONOW >= LICZBIE EKSONOW TO BREAK
    #LICZBA INTRONOW + EKSONOW <= 3 TO BREAK


#dodac tabelke tak by mowila ile bylo odczytow na inpucie, ile przeszło pozytywnie a ile zostało przerwanych w trakcie
#dodatkowa tabelka ze wszystkimi eksonami które byly za krótkie teżhbn

In [59]:
def cutting_scrap(path_to_file_before_MAFFT, path_to_file_after_MAFFT, acceptable_gap_length):
    data_frames = []
    gff_data_frames = []
    gaps_signs = "-" * acceptable_gap_length #maximum length of gaps in sequence in one exon's sequence
    files_in_progress = 0
    print(f"\n \n \n \n Just run function: cutting_scrap")

    for filename in os.listdir(path_to_file_after_MAFFT):
        #print(f"\n Just tried make 1st turn - find all filenames, \n there is filename: {str(filename)}")
        file = os.path.join(path_to_file_after_MAFFT, filename)
        if not os.path.isfile(file):
            continue
        if file.endswith(".fasta"):
            files_in_progress += 1
            count_files = percentage_of_advancement(path_to_file_before_MAFFT)
            print(f"\n ################################################################################### \n running file: {filename} which is {files_in_progress} of {count_files}, what means {round(files_in_progress/count_files*100, 2)}% of advancement \n ################################################################################### \n")
            alignment = AlignIO.read(file, "fasta")
            alignment_id = alignment[0].id
            alignment_DIDNT_TOUCHED = AlignIO.read(file, "fasta")
            #print(f"NA POCZATKU alignment: {alignment[0].id} ma dlugosc {len(alignment[0])}")

        count_of_errors = 0


        #Pawel mowil o mozliwosci stworzenia slownika w ktorym bylo przypisanie 
        #sekwencja_z_pliku_przed_mafftem do sekwencja_z_pliku_po_maffcie. 
        #Ja nie zrobilem slownika, a wydaje mi sie jeszcze krotsza droga:
        file_before_MAFFT = os.path.join(path_to_file_before_MAFFT, filename)
        if not os.path.isfile(file_before_MAFFT):
            continue

        
        seq1_DT, seq2_DT = extracting_strands_from_alignment(alignment_DIDNT_TOUCHED) #type is Bio.Seq.Seq but works like a string
        #dzialamy teraz nie na iterowanym alignmencie tylko na sekwencji ktora zachowuje sie jak string
        
        nucleotides = ["a", "t", "g", "c", "A", "T", "G", "C"]
        index = 0

        #EMPTY LIST TSV
        id = []
        exon_or_intron_number = []
        is_it_intron = []
        class_of_exon = []
        percent_of_homology_for_exons = []
        length = []
        first_nt_position = []
        last_nt_position = []
        first_10_nt = []
        last_10_nt = []
        path = []
        sequence = []

        #EMPTY LIST GFF
        seqid = [] #name of the chromosome or scaffold; chromosome names can be given with or without the 'chr' prefix. Important note: the seq ID must be one used within Ensembl, i.e. a standard chromosome name or an Ensembl identifier such as a scaffold ID, without any additional content such as species or assembly. See the example GFF output below.
        source = [] #tutaj pominiemy ten krok - damy kropke #Describes the algorithm or the procedure that generated this feature. Typically Genescane or Genebank, respectively. #scaffold backbone trinity  i te numerki jakieś
        typ = [] #Describes what the feature is (mRNA, domain, exon, etc.). #scaffold backbone trinity  i te numerki jakieś #intron lub exon
        start = []
        end = []
        score_list = [] #Typically E-values for sequence similarity and P-values for predictions. Homology there
        strand = []
        phase = [] #Indicates where the feature begins with reference to the reading frame. The phase is one of the integers 0, 1, or 2, indicating the number of bases that should be removed from the beginning of this feature to reach the first base of the next codon.. Propably 0
        attributes = [] #A semicolon-separated list of tag-value pairs, providing additional information about each feature. Some of these tags are predefined, e.g. ID, Name, Alias, Parent . You can see the full list [here](https://github.com/The-Sequence-Ontology/Specifications/blob/master/gff3.md).

        try:
            seq1, seq2, count_of_errors = cleaning_gaps_from_both_edges(seq1_DT, seq2_DT, count_of_errors, alignment, filename) #deleting gaps from 5' and 3'. It is important. The next function start counting from possible exon.
        except ValueError as e:
            print(e)
            count_of_errors += 1


        #linking pre-exon's nucleotides (nt-nt pairs) and pre-intron's nucleotides (gap-nt pairs) to theirs indices 
        temp_intron, temp_exon, count_of_errors = indices_of_introns_and_exons_fun(seq1_DT, seq2_DT, seq1, seq2, count_of_errors)
        # print(f"temp_introns indices: {temp_intron}")
        # print(f"temp_exon indices: {temp_exon}")


        #linking indices into single strand of possible exons
        all_exon_range_dict, count_of_errors = start_and_end_parameters_of_exons_dict_fun(temp_exon, gaps_signs, count_of_errors)
        # for key in all_exon_range_dict: #shows exon's sequences according to exon range dict
        #     print(f"Dlugosc eksonu: {all_exon_range_dict[key] - key} ekson: {seq1[key:all_exon_range_dict[key]]}")
        print(all_exon_range_dict)
        #linking indices into single strand of possible introns
        intron_range_dict, count_of_errors  = start_and_end_parameters_of_introns_dict_fun(all_exon_range_dict, count_of_errors)
        # for key in intron_range_dict: #shows introns's sequences according to exon range dict
        #     print(f"Koniec intronu {intron_range_dict[key]} - poczatek intronu{key} = dlugosc intronu {intron_range_dict[key] - key} lista intronow: {seq1[key:intron_range_dict[key]]} \n sekwencja genomowa odpowiadajaca pozycjami wobec intronu:  {seq2[key:intron_range_dict[key]]} \n")


        #sorting exons to propoer categories
        fine_exons, TLH_exons, TS_exons, TLH_percentage_of_identity, fine_percentage_of_identity, fine_exons_range_dict, TS_exon_range_dict, TLH_exon_range_dict = making_and_sorting_alignments_fun(all_exon_range_dict, min_length_aligned_sequence, seq1, seq2)
        #print(f"\n all fine exons = {fine_exons}, \n too low homology = {TLH_exons}, \n too short exons = {TS_exons}")
        print(f" fine_exons_range_dict: {fine_exons_range_dict}, \n too short exon range dict {TS_exon_range_dict}, \n too low homology exon range dict {TLH_exon_range_dict}")


        for element in range(len(fine_exons)):
            print(f"alignment: {alignment[0].id}: and his all fine exon sequence: \n {fine_exons[element]} \n")
            #printing sequence of fine exons

        #gff table for fine exons
        seqid, source, typ, score_list, start, end, strand, phase, attributes = making_exon_gff_table(fine_exons, seqid, source, typ, score_list, start, end, strand, phase, attributes, alignment_id, fine_percentage_of_identity, fine_exons_range_dict)

        #for introns we made other functions with similiar output. Then two tables would be merged and sorted by increasingvalues

        ####################################################################################################################
        ####################################################################################################################
        ####################################################################################################################
        ####################################################################################################################
        ####################################################################################################################

        
        print('\n \n \n WIELKIE PODSUMOWANIE: \n\n\n')
        print(f"poczatki dla fine: {fine_exons_range_dict.keys()}, \n konce: {fine_exons_range_dict.values()}")
        print(f"seqid: \n {seqid}, \n source \n {source},  \n typ: {typ},  \n score_list: {score_list}, \n strand: {strand}, \n phase: {phase}, \n attributes: {attributes}, \n start: {start}, \n end: {end}")

        print(intron_range_dict)
            

#############################################################################################################################
#######################################                     FUNKCJE                   #######################################       
#############################################################################################################################

def percentage_of_advancement(directory):
    count = 0
    for file in os.listdir(directory):
        if file.endswith(".fasta"):
            count += 1
    return(count)
    

def cleaning_gaps_from_both_edges(sequence_transcipt, sequence_genome, count_of_errors, alignment, filename):
    sequence_transcipt_left_shorted = sequence_transcipt.lstrip("-")
    count_deleted_gaps_left = len(sequence_transcipt) - len(sequence_transcipt_left_shorted) #inform how many gaps were deleted from 5'
    
    sequence_transcipt_right_and_left_shorted = sequence_transcipt_left_shorted.rstrip("-")
    count_deleted_gaps_right = len(sequence_transcipt_left_shorted) - len(sequence_transcipt_right_and_left_shorted) #inform how many gaps were deleted from 3'
    
    sequence_genome_left_and_right_shorted = sequence_genome[count_deleted_gaps_left:-count_deleted_gaps_right]
    if len(sequence_genome_left_and_right_shorted) != len(sequence_transcipt_right_and_left_shorted):
        raise ValueError(f" VALUE ERROR: Given sequences: {alignment[0].id} - len: {len(alignment[0])} and  {alignment[1].id} - len {len(alignment[0].id)} from {filename} must have the same length.")
    return sequence_transcipt_right_and_left_shorted, sequence_genome_left_and_right_shorted, count_of_errors


def indices_of_introns_and_exons_fun(seq1_DT, seq2_DT, seq1, seq2, count_of_errors):
    #linking pre-exon's nucleotides (nt-nt pairs) and pre-intron's nucleotides (gap-nt pairs) to theirs indices 
    #NOTE: output is nucleotide or gap from seq1 (transcript sequence). Even if it is gap like this: 
    #seq1: aaatttggg, seq2: aaa---ggg, output will be 'ttt' instead '---'
    #it shows that where were gap, seq2 genome or in seq1 transcript
    temp_exon = [] #list containing indices of intron's positions in sequence.
    temp_intron = [] #list containing indices of intron's positions in sequence.
    index = 0
    paired_nucleotides = zip(seq1, seq2)
    
    if len(seq1_DT) != len(seq2_DT):
        raise ValueError("The two sequences must be of the same length.")
        #return count_of_errors
    for nt1, nt2 in paired_nucleotides:
        if "-" in (nt1, nt2):
            temp_intron.append(index)
        else:
            temp_exon.append(index)
        index += 1 #indeks do wskazywania pozycji w sekwencji 
    return temp_intron, temp_exon, count_of_errors


def start_and_end_parameters_of_exons_dict_fun(temp_exon, gaps_signs, count_of_errors):
    #linking indices into single strand of possible exons
    start_exon = temp_exon[0]
    all_exon_range_dict = {} #that dictionary contains: key(index of start exon) and value(index of end exon)
    
    for i in range(len(temp_exon)): 
        #print(start_exon)
        if (temp_exon[i] - temp_exon[i-1]) > 1+len(gaps_signs): #if difference between two indices of exons's positions is bigger than given number (2), that smaller number is index of 3' nucleotide in exon 
            end_exon = temp_exon[i-1] #temp_exon[i-1] because it is index in list temp_exon. +1 because python starts counting from 0
            all_exon_range_dict[start_exon] = end_exon #creating dictionary with all exons, even with theese too short and with too low homology
            start_exon = temp_exon[i] + 1
            #brakuje ostatniego eksonu. Jest to spowodowane tym, ze nie mozna okreslic parametru end_exon, gdyz nie w liscie temp_exon wiekszego numeru niz ten ostatni
    all_exon_range_dict[start_exon] = temp_exon[-1]

    if all_exon_range_dict == {}:
        raise ValueError(f"ValueError - all_exon_range_dict seems to be empty. Its length = {len(all_exon_range_dict)}")
    #print(f" \n all_exon_range_dict: {all_exon_range_dict}")
    return (all_exon_range_dict, count_of_errors)


     
def start_and_end_parameters_of_introns_dict_fun(all_exon_range_dict, count_of_errors):
    #linking indices into single strand of possible introns
    intron_range_dict = {}
    keys_from_all_exon_range_dict = sorted(all_exon_range_dict.keys()) #list that contains exons's start positions
    #print(f"keys_from_all_exon_range_dict: {keys_from_all_exon_range_dict}\n")

    for i in range(len(keys_from_all_exon_range_dict) - 1):
        start_intron = all_exon_range_dict[keys_from_all_exon_range_dict[i]] +1 
        end_intron = keys_from_all_exon_range_dict[i + 1] - 1
        intron_range_dict[start_intron] = end_intron
    if intron_range_dict == {}:
        raise ValueError(f"ValueError - intron_range_dict seems to be empty. Its length = {len(intron_range_dict)}")
    #print(f" \n intron_range_dict: {intron_range_dict}")
    return (intron_range_dict, count_of_errors)


#zmienione all_fine_exons na fine_exons
def making_and_sorting_alignments_fun(all_exon_range_dict, min_length_aligned_sequence, seq1, seq2):
    fine_exons_range_dict = {}
    TS_exon_range_dict = {} #too
    TLH_exon_range_dict = {}
    TS_exons = []
    TLH_exons = []
    fine_exons = []
    fine_percentage_of_identity = []
    TLH_percentage_of_identity = []
    for key in all_exon_range_dict:
        query = seq1[key-1:all_exon_range_dict[key]+1]
        target = seq2[key-1:all_exon_range_dict[key]+1]
        if len(target) > min_length_aligned_sequence:
            aligner = Align.PairwiseAligner()
            aligner.mismatch_score = 0 #customized setting towards get pure percentage of homology, not alignment with gap penalty etc. The object of interest is simply that how many nt in query has equivalend in target seq. 
            aligner.open_gap_score = 0
            aligner.extend_gap_score = 0
            score = aligner.score(query, target)
            identity_level = (round(((score * 100) / len(target)), 2))
            #print(f"query: {query}, \n target: {target}, percent of homology = {round(score / len(target), 2) * 100} %, score = {score}, len = {len(target)}\n")
            if score >= extreme_homology * 100:
                fine_exons.append(query)
                fine_exons_range_dict[key] = all_exon_range_dict[key]
                fine_percentage_of_identity.append(identity_level)
                #print(f"query: {query} goes to fine_exons")
            else:
                TLH_percentage_of_identity.append(identity_level)
                TLH_exons.append(query)
                TLH_exon_range_dict[key] = all_exon_range_dict[key]       
                #print(f"query: {query} goes to TLH_exons")
        else:
            TS_exons.append(query)
            TS_exon_range_dict[key] = all_exon_range_dict[key]       
            #print(f"query: {query} goes to TS_exons")
    return fine_exons, TLH_exons, TS_exons, TLH_percentage_of_identity, fine_percentage_of_identity, fine_exons_range_dict, TS_exon_range_dict, TLH_exon_range_dict

def extending_source_data_frame(alignment_id, source):
    #print(f"Alignment: {alignment[0].id}, funkcja: extending_source_data_frame, przeszlo")
    keywords = ["TRINITY", "BACKBONE", "SCAFFOLD"]
    found = False
    for key in keywords:
        alignment_id = str(alignment_id).lower()
        if alignment_id.find(key.lower()) != -1:
            source.append(key)
            found = True
            break
    if not found:
        source.append(None) #none means unknown
    return source


def extending_strand_data_frame(alignment_id, strand): 
    #print(f"Alignment: {alignment[0].id}, funkcja: extending_strand_data_frame, przeszlo")
    if str(alignment_id).find("SL+") != -1:
        strand.append("+")
    elif str(alignment_id).find("SL-") != -1:
        strand.append("-")
    else:
        strand.append(None)
    return strand


def appending_list_by_start_exon_position(dict_with_exon_range, start, element):
    #for all types of exons in table
    sorted_keys_start_exon = sorted(dict_with_exon_range.keys())
    start.append(sorted_keys_start_exon[element])
    return start

def appending_list_by_end_exon_position(dict_with_exon_range, end, element):
    #for all types of exons in table
    sorted_values_end_exon = sorted(dict_with_exon_range.values())
    end.append(sorted_values_end_exon[element])
    return end

def making_exon_gff_table(fine_exons, seqid, source, typ, score_list, start, end, strand, phase, attributes, alignment_id, fine_percentage_of_identity, fine_exons_range_dict):
    for element in range(len(fine_exons)):
        print("\n \n teraz analizujemy takie sekwencje: \n", fine_exons[element])
        seqid.append(alignment_id) #what organism
        source = extending_source_data_frame(alignment_id, source) #backbone, trinity or scaffold. Usually trinity
        typ.append("exon") #exon or trinity
        score_list.append(fine_percentage_of_identity[element]) #doesn't matter for Ania
        start = appending_list_by_start_exon_position(fine_exons_range_dict, start, element)
        end = appending_list_by_end_exon_position(fine_exons_range_dict, end, element)
        strand = extending_strand_data_frame(alignment_id, strand) # + or - from sequence's name 
        phase.append(".") #????
        attributes.append(alignment_id) #other
    return seqid, source, typ, score_list, start, end, strand, phase, attributes



#zmodyfikuj ta funkcje by byla fajna
def making__intron_gff_table(intron_range_dict, seqid, source, typ, score_list, start, end, strand, phase, attributes, alignment_id):
    for element in range(len(fine_exons)):
        print("\n \n teraz analizujemy takie sekwencje: \n", fine_exons[element])
        seqid.append(alignment_id) #what organism
        source = extending_source_data_frame(alignment_id, source) #backbone, trinity or scaffold. Usually trinity
        typ.append("exon") #exon or trinity
        score_list.append(fine_percentage_of_identity[element]) #doesn't matter for Ania
        start = appending_list_by_start_exon_position(fine_exons_range_dict, start, element)
        end = appending_list_by_end_exon_position(fine_exons_range_dict, end, element)
        strand = extending_strand_data_frame(alignment_id, strand) # + or - from sequence's name 
        phase.append(".") #????
        attributes.append(alignment_id) #other
    return seqid, source, typ, score_list, start, end, strand, phase, attributes


#nowe
#################################
#stare




def extracting_strands_from_alignment(alignment): #that function describe what strand is transcript strand and genomic strand - in our files, transcript strand has longer ID.
    #print(f"Alignment: {alignment[0].id}, funkcja: extracting_strands_from_alignment, przeszlo")
    if len(alignment[0].id) >= len(alignment[1].id):
        seq1 = alignment[0].seq
        seq2 = alignment[1].seq
        #seq1_is_transcript = True #obecnie nie uzywane
    else:
        seq1 = alignment[1].seq
        seq2 = alignment[0].seq    
        #seq1_is_transcript = False #obecnie nie uzywane
    #print(f"\n Just run function: extracting_strands_from_alignment")
    return seq1, seq2
    #return seq1_is_transcript #obecnie nie uzywane
    

def finding_gap_index(sequence, acceptable_gap_length, alignment): #finding index of first gap in each sequence
    #print(f"Alignment: {alignment[0].id}, funkcja: finding_gap_index, przeszlo")
    gap_index = sequence.find("-" * acceptable_gap_length) #-1 is output when anything was found
    #print(type(gap_index))
    return gap_index


def reversed_finding_gap_index(sequence, acceptable_gap_length, alignment): #finding index of first gap in each sequence for counting the end of intron
    #print(f"Alignment: {alignment[0].id}, funkcja: reversed_finding_gap_index, przeszlo")
    return sequence.rfind("-" * acceptable_gap_length) + acceptable_gap_length #now we have to add length of gap to index, because python starts counting from 0 nt, and we just change direction of counting.

def setting_distance_between_first_nt_and_gap(is_it_exon, first_gap_position_seq1, first_gap_position_seq2, last_gap_position_seq1, last_gap_position_seq2, seq1, alignment): #it means lenght of exon
    #print(f"Alignment: {alignment[0].id}, funkcja: setting_distance_between_first_nt_and_gap, przeszlo")
    if is_it_exon == True:
        if first_gap_position_seq2 < 0: #when in second strand there are no gaps - it means, second strand is correctly prepared. <0 because find() function gives -1 result when not find
            distance_between_first_nt_and_gap = first_gap_position_seq1 #more important is seq2 because it is genomic sequence, ~reference sequence
        elif first_gap_position_seq1 < 0:
            distance_between_first_nt_and_gap = first_gap_position_seq2
        else:
            distance_between_first_nt_and_gap = min(first_gap_position_seq1, first_gap_position_seq2)
        return distance_between_first_nt_and_gap

    elif is_it_exon == False:
        if last_gap_position_seq2 < 0: #it means, if there is no gaps in second sequention, gap position is gap position in seq1. Additional, rfind() function return "-1" if it won't find any gaps in seq
            distance_between_first_nt_and_gap = (len(seq1) - last_gap_position_seq1) -1
            #print("pierwszy", distance_between_first_nt_and_gap) #checkpoint
        else:
            distance_between_first_nt_and_gap = (len(seq1) - max(last_gap_position_seq1, last_gap_position_seq2))
        return distance_between_first_nt_and_gap
            #print("drugi", distance_between_first_nt_and_gap) #checkpoint

        
def describing_class_of_exon(percent_of_homology, last_nt_exon_index, min_length_aligned_sequence, class_of_exon): #class of exon depends of their homology
    #print(f"Alignment: {alignment[0].id}, funkcja: describing_class_of_exon, przeszlo")
    if percent_of_homology >= 0.9 and last_nt_exon_index >= min_length_aligned_sequence:
        class_of_exon.append("1")
    elif percent_of_homology < 0.9 and percent_of_homology > 0 and last_nt_exon_index < min_length_aligned_sequence:
        class_of_exon.append("2")
    else:
        class_of_exon.append("3")


def exons_indices(is_it_exon, max_genomic_seq, seq2, last_nt_exon_index, first_nt_position, last_nt_position):
    #print(f"Alignment: {alignment[0].id}, funkcja: exons_indices, przeszlo")
    if is_it_exon:
        first_nt_index_of_exon = max_genomic_seq.find(str(seq2[:last_nt_exon_index]).replace("-", ""))
        if first_nt_index_of_exon == -1:
            first_nt_position.append(0)
            last_nt_position.append(0)
            #print(f"That exon: {seq2[:last_nt_exon_index].replace('-', '')} comes from genomic sequence and cannot be found in file_before_MAFFT.")
        else:
            first_nt_position.append(first_nt_index_of_exon + 1)
            last_nt_position.append(first_nt_index_of_exon + len(seq2[:last_nt_exon_index]))
            

def introns_indices(is_it_exon, max_genomic_seq, seq2, last_nt_intron_index, first_nt_position, last_nt_position):
    #print(f"Alignment: {alignment[0].id}, funkcja: introns_indices, przeszlo")
    if is_it_exon: 
        first_nt_index_of_intron = max_genomic_seq.find(str(seq2[:last_nt_intron_index]))
        if first_nt_index_of_intron == -1:
            first_nt_position.append(0)
            last_nt_position.append(0)
        else:
            first_nt_position.append(first_nt_index_of_intron + 1)
            last_nt_position.append(first_nt_index_of_intron + len(seq2[:last_nt_intron_index]))




def extending_start_and_end_data_frame_exons(seq2, seq2_DT, start, end, distance_between_first_nt_and_gap, last_nt_exon_index): #index of exons's start
    #print(f"Alignment: {alignment[0].id}, funkcja: extending_start_and_end_data_frame, przeszlo")
    #print(f"Alignment:, funkcja: extending_start_and_end_data_frame, przeszlo")
    start.append(seq2_DT.find(seq2[:distance_between_first_nt_and_gap])+1) #+1 because python starts counting from 0, not 1
    end.append(seq2_DT.find(seq2[:distance_between_first_nt_and_gap]) + last_nt_exon_index)

def extending_start_and_end_data_frame_introns(seq2, seq2_DT, start, end, distance_between_first_nt_and_gap, last_nt_intron_index): #index of intron's start
    #print(f"Alignment: {alignment[0].id}, funkcja: extending_start_and_end_data_frame, przeszlo")
    #print(f"Alignment:, funkcja: extending_start_and_end_data_frame, przeszlo")
    start.append(seq2_DT.find(seq2[:distance_between_first_nt_and_gap])+1)
    end.append(seq2_DT.find(seq2[:distance_between_first_nt_and_gap]) + last_nt_intron_index)

def save_to_file(final_data_frame, filename):
    #print(f"Alignment: {alignment[0].id}, funkcja: save_to_file, przeszlo")
    if os.path.isfile(filename):
        user_input = input("TSV file already exists. Do you want overwrite? y/n: ")
        if user_input == "n":
            base_name, ext = os.path.splitext(filename)
            i = 1
            while os.path.isfile(f"{base_name}_{i}{ext}"):
                i += 1
            filename = f"{base_name}_{i}{ext}"
            final_data_frame.to_csv(filename, sep = "\t")
            
        elif user_input == "y":
            final_data_frame.to_csv(filename, sep = "\t")
        else:
            print("Clarify your answer. Nothing has done.")
            
    else:
        final_data_frame.to_csv(filename, sep = "\t")

def save_to_gff_file(gff_final_data_frame, filename):
    #print(f"Alignment: {alignment[0].id}, funkcja: save_to_gff_file, przeszlo")
    if os.path.isfile(filename):
        user_input = input("GFF file already exists. Do you want overwrite? y/n: ")
        if user_input == "n":
            base_name, ext = os.path.splitext(filename)
            i = 1
            while os.path.isfile(f"{base_name}_{i}{ext}"):
                i += 1
            filename = f"{base_name}_{i}{ext}"
            gff_final_data_frame.to_csv(filename, sep = "\t")
            
        elif user_input == "y":
            gff_final_data_frame.to_csv(filename, sep = "\t")
        else:
            print("Clarify your answer. Nothing has done.")
            
    else:
        gff_final_data_frame.to_csv(filename, sep = "\t")

def check_alignment_eligibility(alignment, alignment_DIDNT_TOUCHED, acceptable_gap_length, files_in_progress, count_files, count_of_errors, filename, extreme_homology, min_length_aligned_sequence):
    while True:
        is_it_exon = True
        first_nucleotides_pair = alignment[:, 0]#first nucleotides in both strands as variable
        if "-" in first_nucleotides_pair: #deleting mismatches at the start
            if len(alignment[0]) < 100: #zakoczenie przycinania - nie wiem czy to jest to o co mi chodzilo
                break
            else:
                alignment = alignment[:, 1:]
                #print(f"alignment after cut:\n{alignment}") #chekpoint
                if len(alignment[0]) % 10 == 0:
                    # print(f"{round((100-(len(alignment[0])/len(alignment_DIDNT_TOUCHED[0]))*100), 2)}% base pair done of {alignment[0].id}. {files_in_progress} of {count_files}, what means {round(files_in_progress/count_files*100, 2)}% of advancement. The time is: {time.strftime('%H:%M:%S', time.localtime())}")
                    # print(f"First 50 nt of reference sequence is: {(alignment[1].seq)[:50]}. \n")
                    function_progress(alignment, alignment_DIDNT_TOUCHED, files_in_progress, count_files)
            continue
    
                
        seq1, seq2 = extracting_strands_from_alignment(alignment)
        seq1_DT, seq2_DT = extracting_strands_from_alignment(alignment_DIDNT_TOUCHED) #sequences without cutted gaps from the ends. They will be used to count index where intron starts
        
    
        first_gap_position_seq1 = finding_gap_index(seq1, acceptable_gap_length, alignment)
        first_gap_position_seq2 = finding_gap_index(seq2, acceptable_gap_length, alignment)
        last_gap_position_seq1 = reversed_finding_gap_index(seq1, acceptable_gap_length, alignment)
        last_gap_position_seq2 = reversed_finding_gap_index(seq2, acceptable_gap_length, alignment)
        #print(f"first gap position in first sequence:{first_gap_position_seq1}, in 2nd sequence: {first_gap_position_seq2}") #checkpoint
    
        distance_between_first_nt_and_gap = setting_distance_between_first_nt_and_gap(is_it_exon, first_gap_position_seq1, first_gap_position_seq2, last_gap_position_seq1, last_gap_position_seq2, seq1, alignment)
        
        local_alignment = pairwise2.align.localxx(seq1[:distance_between_first_nt_and_gap], seq2[:distance_between_first_nt_and_gap], one_alignment_only = True) #"xx"  means no gap penalty while opening gaps or longering them and no penalties for mismatch. Just pure score of alignment to count homology
        print("local_alignment:", local_alignment) #checkpoint
    
        try:  #instruction what to do when seq is too short
            local_homology_percentage = (local_alignment[0].score / distance_between_first_nt_and_gap) 
        except IndexError:
            if len(seq1) < 100:
                count_of_errors += 1
                print(f"That file {filename} made {count_of_errors}th error")
                #print(f"\n \n {alignment[0].id} sequence is too short, sequence has not exons or other fault. \n")
                break
            continue

        
        local_homology_percentage = (local_alignment[0].score / distance_between_first_nt_and_gap)
        print(local_homology_percentage)

    
        if local_homology_percentage <= extreme_homology:
            print(f" Too low score of homology. \n alignment: {alignment[:, :50]} \n score: {local_alignment[0].score}, distance: {distance_between_first_nt_and_gap}, percent: {round(local_homology_percentage, 2)*100}%, \n {local_alignment} \n \n \n \n \n")
            alignment = alignment[:, distance_between_first_nt_and_gap:]
            function_progress(alignment, alignment_DIDNT_TOUCHED, files_in_progress, count_files)            
        else:
            if distance_between_first_nt_and_gap <= min_length_aligned_sequence:
                #print(f"\n alignment is too short. \n distance between sequences {distance_between_first_nt_and_gap}, \n alignment: {alignment}, \n score: {local_alignment[0].score}, distance: {distance_between_first_nt_and_gap}, procentowo: {round(local_homology_percentage, 2)*100}%, \n {local_alignment}")
                alignment = alignment[:, distance_between_first_nt_and_gap:]
                function_progress(alignment, alignment_DIDNT_TOUCHED, files_in_progress, count_files)
            else:
                #print(f"Sequence {alignment[0].id} cut properly from LEFT side, \n distance between sequences: {distance_between_first_nt_and_gap}"), alignment: {alignment[:, :50]},
                break
                
        if len(seq1) < 100 or len(seq2) < 100:
            break

        return seq1, seq2, seq1_DT, seq2_DT, first_gap_position_seq1, first_gap_position_seq2, last_gap_position_seq1, last_gap_position_seq2, distance_between_first_nt_and_gap, local_alignment, local_homology_percentage

    ######################### TERAZ OD PRAWEJ DO LEWEJ #######################
    
    while True:
            
        is_it_exon = False
        last_nucleotides_pair = alignment[:, -1]#last nucleotides in both strands as variable

    
        if "-" in last_nucleotides_pair: #deleting mismatches at the end
            if len(alignment[0]) < 100 or len(alignment[1]) < 100:
                #print("alignment has no matching nucleotides")
                break 
            else:
                alignment = alignment[:, :-1]
                function_progress(alignment, alignment_DIDNT_TOUCHED, files_in_progress, count_files)
                #print(f"alignment check:\n{alignment[:, -50:-1]}") checkpoint
            continue
        else:
            if len(alignment[0]) < 100 or len(alignment[1]) < 0:
                #print("counting from right side: alignment is too short")
                break
        
    
        seq1, seq2 = extracting_strands_from_alignment(alignment)
        seq1_DT, seq2_DT = extracting_strands_from_alignment(alignment_DIDNT_TOUCHED) #sequences without cutted gaps from the ends. They will be used to count index where intron starts
        

        first_gap_position_seq1 = finding_gap_index(seq1, acceptable_gap_length, alignment)
        first_gap_position_seq2 = finding_gap_index(seq2, acceptable_gap_length, alignment)
        last_gap_position_seq1 = reversed_finding_gap_index(seq1, acceptable_gap_length, alignment)
        last_gap_position_seq2 = reversed_finding_gap_index(seq2, acceptable_gap_length, alignment)
        #print("seq1:", last_gap_position_seq1, "seq2:", last_gap_position_seq2) #checkpoint
        
        distance_between_first_nt_and_gap = setting_distance_between_first_nt_and_gap(is_it_exon, first_gap_position_seq1, first_gap_position_seq2, last_gap_position_seq1, last_gap_position_seq2, seq1, alignment)
        #print("drugi", distance_between_first_nt_and_gap)
                
        local_alignment = pairwise2.align.localxx(seq1[-distance_between_first_nt_and_gap:], seq2[-distance_between_first_nt_and_gap:], one_alignment_only = True) #"xx" means no gap penalty while opening gaps or longering them and no penalties for mismatch. Just pure score of alignment
        #print("local_alignment:", local_alignment) #checkpoint
                
    
        try:#instruction what to do when seq is too short or gaps were not found
            if distance_between_first_nt_and_gap != 0:
                local_homology_percentage = (local_alignment[0].score / distance_between_first_nt_and_gap) 
            else:
                break
        except IndexError:
            if len(seq1) < 100:
                count_of_errors += 1
                print(f"That file {filename} made {count_of_errors}th error")
                #print(f"\n \n {alignment[0].id} sequence is too short or sequence has not exons. \n")
                break
            continue
    
        
        if local_homology_percentage <= extreme_homology:
            #print(f"\n Too low homology! \n alignment: {alignment[:, -50:]} \n score: {local_alignment[0].score}, \distance: {distance_between_first_nt_and_gap}, %%%: {round(local_homology_percentage, 2)*100}%, \n {local_alignment}")
            alignment = alignment[:, :-distance_between_first_nt_and_gap] 
            function_progress(alignment, alignment_DIDNT_TOUCHED, files_in_progress, count_files)
            
        else:
            if distance_between_first_nt_and_gap <= min_length_aligned_sequence:
                #print(f"\n Alignment is too short!. \n distance: {distance_between_first_nt_and_gap}, \n alignment: {alignment}, \n score: {local_alignment[0].score}, distance: {distance_between_first_nt_and_gap}, %%%: {round(local_homology_percentage, 2)*100}%, \n {local_alignment}")
                alignment = alignment[:, :-distance_between_first_nt_and_gap]
                function_progress(alignment, alignment_DIDNT_TOUCHED, files_in_progress, count_files)
            else:
                #print(f"Sequence: {alignment[0].id} cut properly from RIGHT side, alignment: \n {alignment[:, -50:]}")
                break
                
        return seq1, seq2, seq1_DT, seq2_DT, first_gap_position_seq1, first_gap_position_seq2, last_gap_position_seq1, last_gap_position_seq2, distance_between_first_nt_and_gap, local_alignment, local_homology_percentage

        if len(seq1) < 100 or len(seq2) < 100:
            break

def function_progress(alignment, alignment_DIDNT_TOUCHED, files_in_progress, count_files):
    #do tego tematu wrocimy jak zrobimy tabele z odrzuconymi wartosciami
    first_50_nt_alignment = str((alignment[1].seq)[:20])
    #print(f" {round((100-(len(alignment[0])\/len(alignment_DIDNT_TOUCHED[0]))*100), 2)}% % base pair done of {alignment[0].id}. {files_in_progress} of {count_files}, what means {round(files_in_progress/count_files*100, 2)}% of advancement. The time is: {time.strftime('%H:%M:%S', time.localtime())}")

if __name__ == '__main__':
    cutting_scrap(path_to_file_before_MAFFT, path_to_file_after_MAFFT, 2)


 
 
 
 Just run function: cutting_scrap

 ################################################################################### 
 running file: LON_OG0030309.fasta which is 1 of 9, what means 11.11% of advancement 
 ################################################################################### 

{0: 112, 695: 920, 1350: 1586, 1904: 2056, 2141: 2222, 3188: 3252, 3546: 3684, 4033: 4126, 4878: 5096, 5866: 6045}
 fine_exons_range_dict: {695: 920, 1350: 1586, 1904: 2056, 3546: 3684, 4878: 5096, 5866: 6045}, 
 too short exon range dict {0: 112}, 
 too low homology exon range dict {2141: 2222, 3188: 3252, 4033: 4126}
alignment: LON_SL+_GGOE01000359.1: and his all fine exon sequence: 
 accgtcaagtgcctgtcaaccgattccgttggcaaccgccttgagcgattgcaacagaccaatgcatcgctggcccgggagatggcagcgctgcgggagcgggcggagtacctggacgtgatggaggagatgaagtacttctcggagcggcccatgacccccatcagccacgctgtgttccacccctttgccccgcacctgacccacaagactgcgttgttgcaggg 

alignment: LON_SL+_GGOE01000359.1: and his all fine exon sequence: 
 ggccctcttc

In [53]:
'''
stan rzeczy: cos sie dubluje, niektore fragmenty tekstu. Mozliwe ze to klopot z iteracja po iteracji. Napisz najpierw slownik

cel 0: zmien frament na poczatku zeby nie iterowal po iterowanym tekscie. prostsza metoda, nie slownik 
CHYBA GIT

cel: usprawnij to tak by outputem byl 148 wierszowa tabela
    Dwa klopoty:
        1. wszystkie dane sa od tego samego organizmu --> klopot z nazwa w seq id --> trzeba zmienic sposob laczenia list do slownika UPDATE. to nawet nie jest przyczyna trudności, tu chodzi o kwestie iteracji
        2. niektore dane maja za niska homologie - mozliwe ze sa powiazania z powyzszym punktem
        3. a inne nie respectuja minimalnej dlugosci eksonu

cek na potem: usunac introny z ostatniej pozycji w tabeli - powiazanie z powyzszymi punktami

cel na potem: zrob tak by wyswietlal sie postep na biezaco, nie tylko gdy pierwszym znakiem bedzie gap
    Ta funkcja jest zle skonstruowana. Musimy przyjac za punkt odniesienia stopien zaawansowania w odniesieniu do funkcji Didnt Touched
    Wroce do tego potem jak bede pisał tabele z odrzuconymi danymi 
'''



'\nstan rzeczy: cos sie dubluje, niektore fragmenty tekstu. Mozliwe ze to klopot z iteracja po iteracji. Napisz najpierw slownik\n\ncel 0: zmien frament na poczatku zeby nie iterowal po iterowanym tekscie. prostsza metoda, nie slownik \nCHYBA GIT\n\ncel: usprawnij to tak by outputem byl 148 wierszowa tabela\n    Dwa klopoty:\n        1. wszystkie dane sa od tego samego organizmu --> klopot z nazwa w seq id --> trzeba zmienic sposob laczenia list do slownika UPDATE. to nawet nie jest przyczyna trudności, tu chodzi o kwestie iteracji\n        2. niektore dane maja za niska homologie - mozliwe ze sa powiazania z powyzszym punktem\n        3. a inne nie respectuja minimalnej dlugosci eksonu\n\ncek na potem: usunac introny z ostatniej pozycji w tabeli - powiazanie z powyzszymi punktami\n\ncel na potem: zrob tak by wyswietlal sie postep na biezaco, nie tylko gdy pierwszym znakiem bedzie gap\n    Ta funkcja jest zle skonstruowana. Musimy przyjac za punkt odniesienia stopien zaawansowania w od